In [1]:
import numpy as np
import random
import scipy.linalg
import gym
from Utility import DerivativeLiftFunc, data_collecter

/home/stone/anaconda3/envs/mujoco/lib/python3.7/site-packages/ale_py/roms/utils.py:90: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for external in metadata.entry_points().get(self.group, []):


In [2]:
ENV = ["DampingPendulum","Pendulum-v1","CartPole-v1","MountainCarContinuous-v0"]

In [3]:
def evalKoopmanD(env_index):
    env_name = ENV[env_index]
    print(env_name)
    Data_collecter = data_collecter(env_name)
    Nstate = Data_collecter.Nstates
    udim = Data_collecter.udim
    LiftFunc = DerivativeLiftFunc(env_name,Nstate,udim)
    NKoopman = LiftFunc.NKoopman
    Samples = 5000
    Steps = 30
    random.seed(1)
    np.random.seed(1)
    def A_and_G_o(s_1, s_2, u): # Uses measurements s(t_k) & s(t_{k+1}) to calculate A and G
        A = np.dot(LiftFunc.Psi_su(s_2, u).reshape(-1,1), LiftFunc.Psi_su(s_1, u).reshape(1,-1))
        # print(A.shape)
        G = np.dot(LiftFunc.Psi_su(s_1, u).reshape(-1,1), LiftFunc.Psi_su(s_1, u).reshape(1,-1))
        return A, G

    np.random.seed(1)
    env = Data_collecter.env
    env.reset()
    Samples = 20000
    A = np.zeros((NKoopman+1, NKoopman+1))
    G = np.zeros((NKoopman+1, NKoopman+1))

    Ps0_list = np.empty((Samples,NKoopman+1))
    Psi_list = np.empty((Samples,NKoopman+1))

    for i in range (Samples):

        # Sample states
        s0 = Data_collecter.random_state()
        u10 = np.random.uniform(Data_collecter.umin, Data_collecter.umax)

        # Simulate system forward
        env.reset_state(s0)
        sn = env.step(u10)
        sn = sn[0]
        # sn = odeint(single_pendulum, s0, [0, ts], args=(u10,))
        # sn = sn[-1,:]   

        # Evaluate basis functions at t = 0 and t = ts
        Ps0_list[i,:] = LiftFunc.Psi_su(s0, u10).reshape(-1)
        Psi_list[i,:] = LiftFunc.Psi_su(sn, u10).reshape(-1)

        [Atemp, Gtemp] = A_and_G_o(s0,sn,u10)
        A = A+Atemp
        G = G+Gtemp

    Kd = np.dot(A,scipy.linalg.pinv2(G)) # more accurate than numpy
    # print("The identified Koopman operator is \n", Kd)
    eig = np.linalg.eigvals(Kd)
    print("max eig val:{}".format(max(eig)))

    ## Measure maximum local (across one time step) errors in Ψ(s_{k+1}) - Kd*Ψ(s_k)
    local_errors = np.empty([Samples, NKoopman+1])
    for i in range(Samples):
        local_errors[i,:] = abs(Psi_list[i,:]- np.dot(Kd,Ps0_list[i,:]))
    max_local_errors = np.amax(local_errors, axis = 0)
    print('Max local errors in theta: {}'.format(max_local_errors[:Nstate]))
    # mean_local_errors = np.mean(local_errors, axis = 0)
    # print('Mean local errors in theta: {}'.format(mean_local_errors[:Nstate]))# Save trained Koopman and basis functions measurements used to obtain it
# io.savemat('Data - Koopman_and_BasisFunctions_CartPole.mat', {'Kd' : Kd, 'max_local_errors' : max_local_errors}) # save variables to Matlab file
    # train_data = Data_collecter.collect_koopman_data(20000,1)
    # X_lift = np.zeros((20000,NKoopman+udim))
    # Y_lift = np.zeros((20000,NKoopman+udim))
    # for i in range(20000):
    #     X_lift[i] = LiftFunc.Psi_su(train_data[0,i,udim:],train_data[0,i,:udim])
    #     Y_lift[i] = LiftFunc.Psi_su(train_data[1,i,udim:],train_data[0,i,:udim])
    # XX = np.dot(X_lift.T,X_lift)
    # XY = np.dot(X_lift.T,Y_lift)
    # Kd = np.dot(scipy.linalg.pinv2(XX),XY).T
    # eig = np.linalg.eigvals(Kd)
    # print("max eig val:{}".format(max(eig)))
    np.save("Prediction_Results/"+"Kd_"+env_name+"_KoopmanDerivative"+".npy",Kd)  
    # local_errors = abs(Y_lift- np.dot(X_lift,Kd.T))
    # max_local_errors = np.amax(local_errors, axis = 0)
    # print("max err:",max_local_errors)
    # random.seed(2022)
    # np.random.seed(2022)
    times = 4 
    max_loss_all = np.zeros((times,Steps))
    mean_loss_all = np.zeros((times,Steps))
    for t in range(times):
        test_data = Data_collecter.collect_koopman_data(Samples,Steps)
        X_current = test_data[0,:,:]
        X_current_lift = np.zeros((Samples,NKoopman+udim))
        maxloss_list = []
        meanloss_list = []
        for i in range(Samples):
            X_current_lift[i] = LiftFunc.Psi_su(X_current[i,udim:],X_current[i,:udim])
        for i in range(Steps):
            X_current_lift = np.dot(X_current_lift,Kd.T)
            X_current_lift[:,NKoopman:] = test_data[i+1,:,:udim]
            Err = X_current_lift[:,:Nstate] - test_data[i+1,:,udim:]
            maxloss = np.mean(np.max(abs(Err),axis=0))
            meanloss = np.mean(np.mean(abs(Err),axis=0))
            maxloss_list.append(maxloss)
            meanloss_list.append(meanloss)
        max_loss_all[t] = np.array(maxloss_list).reshape(-1)
        mean_loss_all[t] = np.array(meanloss_list).reshape(-1)
    max_mean = np.mean(max_loss_all,axis=0)
    max_std = np.std(max_loss_all,axis=0)
    mean_mean =  np.mean(mean_loss_all,axis=0)
    mean_std =  np.std(mean_loss_all,axis=0)  
    np.save("Prediction_Results/"+env_name+"_KoopmanDerivative"+".npy",np.array([max_mean,max_std,mean_mean,mean_std]))
    return max_mean,max_std,mean_mean,mean_std 

In [4]:
for j in range(4):
     evalKoopmanD(j)

DampingPendulum


/home/stone/anaconda3/envs/mujoco/lib/python3.7/site-packages/ipykernel_launcher.py:50: DeprecationWarning: scipy.linalg.pinv2 is deprecated since SciPy 1.7.0, use scipy.linalg.pinv instead


max eig val:(0.9999999999999993+0j)
Max local errors in theta: [0.00159037 0.15856233]
Pendulum-v1
max eig val:(0.9999999999999982+0j)
Max local errors in theta: [4.61852778e-14 7.99360578e-14]
CartPole-v1


/home/stone/Documents/Koopman/gym/gym/envs/classic_control/cartpole.py:156: UserWarning: WARN: You are calling 'step()' even though this environment has already returned done = True. You should always call 'reset()' once you receive 'done = True' -- any further steps are undefined behavior.
  "You are calling 'step()' even though this "


max eig val:1.0671169829832712
Max local errors in theta: [2.38088098e-07 1.49414054e-03 3.00818355e-08 1.83861029e-02]
MountainCarContinuous-v0
max eig val:(1.0000000000000888+0j)
Max local errors in theta: [2.99026564e-08 2.77465595e-08]


In [2]:
env_name = "DampingPendulum"
Data_collecter = data_collecter(env_name)
Nstate = Data_collecter.Nstates
udim = Data_collecter.udim
LiftFunc = DerivativeLiftFunc(env_name,Nstate,udim)
NKoopman = LiftFunc.NKoopman
Samples = 5000
Steps = 30
train_data = Data_collecter.collect_koopman_data(20000,1)

In [3]:
X_lift = np.zeros((20000,NKoopman+udim))
Y_lift = np.zeros((20000,NKoopman+udim))
for i in range(20000):
    X_lift[i] = LiftFunc.Psi_su(train_data[0,i,udim:],train_data[0,i,:udim])
    Y_lift[i] = LiftFunc.Psi_su(train_data[1,i,udim:],train_data[0,i,:udim])
XX = np.dot(X_lift.T,X_lift)
XY = np.dot(X_lift.T,Y_lift)
Kd = np.dot(scipy.linalg.pinv2(XX),XY).T
local_errors = abs(Y_lift- np.dot(X_lift,Kd.T))
max_local_errors = np.amax(local_errors, axis = 0)
print("max err:",max_local_errors)


max err: [5.25485295e-05 5.39188926e-03 6.79820783e-03 1.38979410e-01
 3.35997896e-12]


/home/stone/anaconda3/envs/mujoco/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: scipy.linalg.pinv2 is deprecated since SciPy 1.7.0, use scipy.linalg.pinv instead
  


In [5]:
test_data = Data_collecter.collect_koopman_data(Samples,Steps)
X_current = test_data[0,:,:]
X_current_lift = np.zeros((Samples,NKoopman+udim))
maxloss_list = []
meanloss_list = []
for i in range(Samples):
    X_current_lift[i] = LiftFunc.Psi_su(X_current[i,udim:],X_current[i,:udim])
for i in range(Steps-1):
    X_current_lift = np.dot(X_current_lift,Kd.T)
    X_current_lift[:,NKoopman:] = test_data[i+1,:,:udim]
    Err = X_current_lift[:,:Nstate] - test_data[i+1,:,udim:]
    maxloss = np.mean(np.max(Err,axis=0))
    meanloss = np.mean(np.mean(Err,axis=0))
    maxloss_list.append(maxloss)
    meanloss_list.append(meanloss)
# print(maxloss_list)